<a href="https://colab.research.google.com/github/NLP4/Enhacing-SPECTER-with-some-extensions/blob/main/Full_text_%2B_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/allenai/specter.git

Cloning into 'specter'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 195 (delta 80), reused 70 (delta 59), pack-reused 75
Receiving objects: 100% (195/195), 316.53 KiB | 7.72 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [ ]:
!wget https://ai2-s2-research-public.s3-us-west-2.amazonaws.com/specter/archive.tar.gz

--2023-04-27 19:11:47--  https://ai2-s2-research-public.s3-us-west-2.amazonaws.com/specter/archive.tar.gz
Resolving ai2-s2-research-public.s3-us-west-2.amazonaws.com (ai2-s2-research-public.s3-us-west-2.amazonaws.com)... 52.92.210.74, 52.92.165.34, 3.5.78.12, ...
Connecting to ai2-s2-research-public.s3-us-west-2.amazonaws.com (ai2-s2-research-public.s3-us-west-2.amazonaws.com)|52.92.210.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 873742818 (833M) [application/x-tar]
Saving to: ‘archive.tar.gz’

archive.tar.gz      100%[===================>] 833.27M  32.0MB/s    in 27s     

2023-04-27 19:12:15 (30.3 MB/s) - ‘archive.tar.gz’ saved [873742818/873742818]



In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"oumaimaregragui","key":"98f2ff74d27102ab393ebb934d047f83"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dillonpulliam/cord19cleaneddata

100% 607M/608M [00:05<00:00, 127MB/s]
100% 608M/608M [00:05<00:00, 118MB/s]


In [ ]:
!unzip cord19cleaneddata.zip

Archive:  cord19cleaneddata.zip
  inflating: covidData.csv           
  inflating: covidDataCleaned.csv    


In [ ]:
import re
import numpy as np
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
metadata_path = '/content/covidDataCleaned.csv'
meta_df = pd.read_csv(metadata_path, dtype={'doi': str})

In [ ]:
meta_df = meta_df[['paper_id','abstract','body_text','title','authors','journal','url']].reset_index(drop=True)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00


Extracting the dates

In [ ]:
import pandas as pd
import re
import numpy as np
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a regular expression to match dates
date_regex = re.compile(r"\d{4}")

# Create a new column for the date
meta_df["date"] = ""

# Process each abstract and extract the date
for i, abstract in enumerate(meta_df["abstract"]):
    if isinstance(abstract, float) and np.isnan(abstract):
        abstract = ""
    doc = nlp(abstract)
    for token in doc:
        if token.pos_ == "NUM" and date_regex.match(token.text):
            date = token.text
            break
    else:
        date = ""
    
    # Store the extracted date in the date column
    meta_df.at[i, "date"] = date if isinstance(date, str) else str(date)

In [ ]:
meta_df = meta_df[meta_df['date'].notnull() & (meta_df['date'] > '2018') & (meta_df['date'] < '2024')]

Generating the embeddings

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# Load the metadata
meta_df = meta_df[['paper_id','abstract','body_text','title','authors','journal','url','date']].reset_index(drop=True)

# Load the SPECTER model and tokenizer
model_name = "allenai/specter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Compute the embeddings in batches
batch_size = 12
num_batches = (len(meta_df) + batch_size - 1) // batch_size

embeddings = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(meta_df))
    batch = meta_df.iloc[start_idx:end_idx]
    inputs = list(batch.apply(lambda row: f"{row['title']} {row['abstract']} {row['body_text']}", axis=1))

    # Tokenize the inputs and pad the sequences
    encoded_inputs = tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors='pt')
    padded_inputs = {k: v.to(model.device) for k, v in encoded_inputs.items()}

    # Compute the embeddings for the batch
    with torch.no_grad():
        outputs = model(**padded_inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    embeddings.append(batch_embeddings)

# Concatenate the embeddings for all batches
embeddings = np.concatenate(embeddings, axis=0)

In [ ]:
def get_label(row):
    if "COVID-19" in row['abstract'] or "coronavirus" in row['abstract'] or "SARS-CoV-2" in row['abstract']:
        return "relevant"
    else:
        return "irrelevant"

meta_df['label'] = meta_df.apply(get_label, axis=1)

Classification Task

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, meta_df['label'], test_size=0.2, random_state=42)

# Train a logistic regression classifier
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  irrelevant       0.78      0.76      0.77        33
    relevant       0.90      0.91      0.90        77

    accuracy                           0.86       110
   macro avg       0.84      0.83      0.84       110
weighted avg       0.86      0.86      0.86       110

